In [3]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import join
import csv
import sys
import jieba
import jieba.posseg as pseg
import re
import jieba.analyse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
print("【訊息】目前的工作目錄:",os.getcwd())
import math

【訊息】目前的工作目錄: /workspace/Randy/17life


In [120]:
csv.field_size_limit(int(sys.maxsize/10000000000))
df=pd.read_csv('./data/deal_final_0605', sep=',', engine='python', encoding='utf-8-sig')

刪除nan的itemName行數 並重置索引

In [107]:
# nanList = list(df['item_name'][df['item_name'].isnull()].index)
# df = df.drop(df.index[nanList])
# df['item_name'][df['item_name'].isnull()]
# df = df.reset_index()

In [121]:
itemName = df['item_name']
symbols = r"[！!\-(－（)）.（+/，#_;:=】？?《》|【\]\[~★♥~↘]"
tmp = [re.sub(symbols," ",str(title)) for title in itemName ]
symbols2 = r"[ \d]+元|[\d.]+折[起]?|\$[ \d]+"
itemNameC = [re.sub(symbols2," ",title) for title in tmp ]

In [130]:
itemNameC = itemNameC[:100]

In [141]:
itemNameToken = [ list(filter(lambda x:x.word!=" ",pseg.lcut(i))) for i in itemNameC ]

In [186]:
itemNameToken

[[pair('柯絲妃', 'nr'),
  pair('兒', 'n'),
  pair('胎盤素', 'n'),
  pair('肌因', 'n'),
  pair('賦活', 'v'),
  pair('身體', 'n'),
  pair('精華', 'a'),
  pair('乳', 'n'),
  pair('10', 'm'),
  pair('入', 'v')],
 [pair('aibo', 'eng'),
  pair('領導者', 'n'),
  pair('立體聲', 'd'),
  pair('智慧', 'nr'),
  pair('藍牙', 'ns'),
  pair('耳機', 'n'),
  pair('麥克風', 'nr'),
  pair('系列', 'q'),
  pair('2', 'm'),
  pair('入', 'v'),
  pair('方案', 'n')],
 [pair('aibo', 'eng'),
  pair('領導者', 'n'),
  pair('立體聲', 'd'),
  pair('智慧', 'nr'),
  pair('藍牙', 'ns'),
  pair('耳機', 'n'),
  pair('麥克風', 'nr'),
  pair('系列', 'q'),
  pair('4', 'm'),
  pair('入', 'v'),
  pair('方案', 'n')],
 [pair('aibo', 'eng'),
  pair('領導者', 'n'),
  pair('立體聲', 'd'),
  pair('智慧', 'nr'),
  pair('藍牙', 'ns'),
  pair('耳機', 'n'),
  pair('麥克風', 'nr'),
  pair('系列', 'q'),
  pair('1', 'm'),
  pair('入', 'v'),
  pair('方案', 'n')],
 [pair('aibo', 'eng'),
  pair('領導者', 'n'),
  pair('立體聲', 'd'),
  pair('智慧', 'nr'),
  pair('藍牙', 'ns'),
  pair('耳機', 'n'),
  pair('麥克風', 'nr'),
  pair('系列',

In [143]:
itemNameTokenN =[ list(filter(lambda x:x.flag=="n",i)) for i in itemNameToken ]

In [184]:
df_codetest = list(np.int32(df.code[:100]))

In [185]:
df_codetest= np.array(df_codetest).reshape(-1,1)
itemNameTokenN = np.array(itemNameTokenN).reshape(-1,1)
np.concatenate((df_codetest,itemNameTokenN), axis=1)

array([[5821,
        list([pair('兒', 'n'), pair('胎盤素', 'n'), pair('肌因', 'n'), pair('身體', 'n'), pair('乳', 'n')])],
       [222, list([pair('領導者', 'n'), pair('耳機', 'n'), pair('方案', 'n')])],
       [222, list([pair('領導者', 'n'), pair('耳機', 'n'), pair('方案', 'n')])],
       [222, list([pair('領導者', 'n'), pair('耳機', 'n'), pair('方案', 'n')])],
       [222, list([pair('領導者', 'n'), pair('耳機', 'n'), pair('方案', 'n')])],
       [222, list([pair('領導者', 'n'), pair('耳機', 'n')])],
       [423,
        list([pair('牌', 'n'), pair('香腸', 'n'), pair('原味', 'n'), pair('蒜味', 'n')])],
       [423,
        list([pair('牌', 'n'), pair('香腸', 'n'), pair('原味', 'n'), pair('蒜味', 'n')])],
       [423,
        list([pair('牌', 'n'), pair('香腸', 'n'), pair('原味', 'n'), pair('蒜味', 'n')])],
       [423,
        list([pair('牌', 'n'), pair('香腸', 'n'), pair('原味', 'n'), pair('蒜味', 'n')])],
       [423,
        list([pair('牌', 'n'), pair('香腸', 'n'), pair('原味', 'n'), pair('蒜味', 'n')])],
       [430,
        list([pair('水林', 'n'), pai

In [175]:
np.concatenate((df_codetest,itemNameTokenN), axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [174]:
pd.DataFrame(np.array([df_codetest,itemNameTokenN]))

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,5821,222,222,222,222,222,423,423,423,423,...,1604,1604,1604,1604,1604,1604,1604,1604,1604,1604
1,"[兒/n, 胎盤素/n, 肌因/n, 身體/n, 乳/n]","[領導者/n, 耳機/n, 方案/n]","[領導者/n, 耳機/n, 方案/n]","[領導者/n, 耳機/n, 方案/n]","[領導者/n, 耳機/n, 方案/n]","[領導者/n, 耳機/n]","[牌/n, 香腸/n, 原味/n, 蒜味/n]","[牌/n, 香腸/n, 原味/n, 蒜味/n]","[牌/n, 香腸/n, 原味/n, 蒜味/n]","[牌/n, 香腸/n, 原味/n, 蒜味/n]",...,"[質感/n, 連帽/n, 羊羔/n, 外套/n, 方案/n]","[質感/n, 連帽/n, 羊羔/n, 外套/n, 方案/n]","[質感/n, 連帽/n, 羊羔/n, 外套/n, 方案/n]","[羽絨/n, 棉/n, 連帽/n, 外套/n, 方案/n]","[羽絨/n, 棉/n, 連帽/n, 外套/n]","[質感/n, 連帽/n, 羊羔/n, 外套/n]","[凹凸/n, 版/n, 衣/n, 方案/n]","[衣/n, 衣/n]","[凹凸/n, 版/n, 衣/n, 方案/n]","[素肌/n, 外線/n, 衣/n, 輕量/n, 款/n, 方案/n]"


KeyError: 'code'